# Game data

Game data seems to be contained in `reader*.zbd` archives. Luckily, they use the same file table at the end of the archive, which we know how to extract.

In [1]:
from pathlib import Path
from struct import Struct, unpack_from

ARCHIVE_FOOTER = Struct("<2I")
ARCHIVE_RECORD = Struct("<2I64s76x")


def extract_archive(data):
    offset = len(data) - ARCHIVE_FOOTER.size
    _, count = ARCHIVE_FOOTER.unpack_from(data, offset)
    for _ in range(count):
        offset -= ARCHIVE_RECORD.size  # walk the table backwards
        start, length, name = ARCHIVE_RECORD.unpack_from(data, offset)
        name = name.rstrip(b"\x00").decode("ascii")
        yield name, data[start : start + length]


data = Path("install/v1.0-us-post/zbd/reader.zbd").read_bytes()
reader = dict(extract_archive(data))
reader_names = list(reader.keys())
print("\n".join(reader_names[:5] + reader_names[-5:]))

vulture_prints.zrd
thor_prints.zrd
supernova_prints.zrd
sunder_prints.zrd
strider_prints.zrd
effects.zrd
dialog.zrd
commonAnim.zrd
camera.zrd
briefing.zrd


In [2]:
all(name.endswith(".zrd") for name in reader.keys())

True

In [4]:
import json

UINT32 = Struct("<I")
FLOAT = Struct("<f")


def read(zrd):
    offset = 0

    def _read_node(zrd):
        nonlocal offset
        node_type, = UINT32.unpack_from(zrd, offset)
        offset += UINT32.size
        
        # uint32
        if node_type == 1:
            value, = UINT32.unpack_from(zrd, offset)
            offset += UINT32.size
            return value
    
        # float
        if node_type == 2:
            value, = FLOAT.unpack_from(zrd, offset)
            offset += FLOAT.size
            return value
        
        # string
        if node_type == 3:
            count, = UINT32.unpack_from(zrd, offset)
            offset += UINT32.size
            value = zrd[offset : offset + count].decode("ascii")
            offset += count
            return value
        
        # list
        if node_type == 4:
            count, = UINT32.unpack_from(zrd, offset)
            offset += UINT32.size
            
            # count is one bigger, because in the code this first item
            # of the list stores the item count
            count -= 1
            
            if count == 0:
                return None
            
            # special case to aid readability
            if count == 1:
                return _read_node(zrd)

            values = [_read_node(zrd) for i in range(count)]
            
            # special munging to turn a list of keys and values into a dict
            is_even = count % 2 == 0
            has_keys = all(isinstance(s, str) for s in values[::2])
            if is_even and has_keys:
                it = iter(values)
                values = dict(zip(it, it))

            return values

        raise ValueError(f"Invalid reader node type {node_type} in zRdrRead()")
    
    ret = _read_node(zrd)
    assert offset >= len(zrd)
    return ret

base_path = Path.cwd() / "reader"
for name, zrd in reader.items():
    # print(name)
    json_path = base_path / name.replace(".zrd", ".json")
    with json_path.open("w", encoding="utf-8") as f:
        json.dump(read(zrd), f, indent=2)

As much as it helped with the 3D data to work from the decompiled source, here's it's the other way around. This data is highly structured, so can easily be analysed. The instructions are extremely difficult to figure out, and most of the deserialisation code seems to be hard-coded/baked in.

## Next up

[The game's internal interpreter files](11-interp.ipynb)